# Capstone Project
# Propective Community College locations in Chicago

Problem-----
a portion of the population of Chicago don't have access to an affordable college experience. This is because of the lack of community colleges in the less populated areas of Chicago.  




Background------
For many prospective students, college is very expensive. Many people consider community colleges as a way of starting their college careers. However there are some cluster of neighborhoods in Chicago which lack a nearbly community college. Students will have to take hours of self or public transportation to reach their community college which is probably located in another part of the city. 

Access of education is a fundamental aspect that no communities should disregard. Through community colleges, students in the neighborhood will get access to more oppurtunities in the field. The overall poverty level can be further reduced as more people will get the oppurtunity to get college degrees and make earnings to support themselves and their families. In addition, the construction of the college will increase the number of jobs for workers and the opening of the college will increase the teaching positions available. In summary, the unemployment rate of the neighborhood should decrease. The clients are the city officials who would license and fund the construction and operation of the college. This positive awarness of a new community college will better improve the education and living conditions of many residents. The officials are looking for ways to improve the city's education system so this new plan of creating colleges in low populated areas might improve the overall living condition in the neighborhood.

Data ----------  the data collected is the locations of the nearby community colleges around one of less populated areas in Chicago. The data includes the distance from the center of the neighborhood and the type of institution. 

The data will be used to create a cluster using Kmeans to determine the two closest location from the neighborhood center to build two new community colleges. The data will be collected from foursquare. First observations are that the many of the nearby colleges are located more than 13km way and not many clusters on the west side of the airport. This means that a college is needed on the west side of the airport. 


In [1]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Folium installed
Libraries imported.


In [62]:
CLIENT_ID = 'SSWCFOUOB0WOECC1NNHRZQAVPMI3EY4UOC1LWS4YFJVTKFEF' # your Foursquare ID
CLIENT_SECRET = 'QVGIRPWYW33KOE403CS0K3C2BSED1ZOCRVJHIX0GICG434QE' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)


Your credentails:
CLIENT_ID: SSWCFOUOB0WOECC1NNHRZQAVPMI3EY4UOC1LWS4YFJVTKFEF
CLIENT_SECRET:QVGIRPWYW33KOE403CS0K3C2BSED1ZOCRVJHIX0GICG434QE


In [63]:
address = ' 5700 S Cicero Ave, Chicago, IL'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

41.7885065 -87.7418255


In [64]:
search_query = 'Community College'
radius = 20000
print(search_query + ' .... OK!')

Community College .... OK!


In [65]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=SSWCFOUOB0WOECC1NNHRZQAVPMI3EY4UOC1LWS4YFJVTKFEF&client_secret=QVGIRPWYW33KOE403CS0K3C2BSED1ZOCRVJHIX0GICG434QE&ll=41.7885065,-87.7418255&v=20180604&query=Community College&radius=20000&limit=30'

In [66]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5d7db379acc5f5002c00e97c'},
 'response': {'venues': [{'id': '49e0bafef964a52064611fe3',
    'name': 'Moraine Valley Community College',
    'location': {'address': '9000 W College Pkwy',
     'lat': 41.69194156067703,
     'lng': -87.83764137298584,
     'labeledLatLngs': [{'label': 'display',
       'lat': 41.69194156067703,
       'lng': -87.83764137298584}],
     'distance': 13375,
     'postalCode': '60465',
     'cc': 'US',
     'city': 'Palos Hills',
     'state': 'IL',
     'country': 'United States',
     'formattedAddress': ['9000 W College Pkwy',
      'Palos Hills, IL 60465',
      'United States']},
    'categories': [{'id': '4bf58dd8d48988d1a2941735',
      'name': 'Community College',
      'pluralName': 'Community Colleges',
      'shortName': 'Community College',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/education/communitycollege_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-156851

In [67]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)


In [68]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,Moraine Valley Community College,Community College,9000 W College Pkwy,US,Palos Hills,United States,NaN,13375,"[9000 W College Pkwy, Palos Hills, IL 60465, U...","[{'label': 'display', 'lat': 41.69194156067703...",41.691942,-87.837641,NaN,60465,IL,49e0bafef964a52064611fe3
1,Moraine Valley Community College Southwest Edu...,Community College,NaN,US,Tinley Park,United States,NaN,13330,"[Tinley Park, IL 60477, United States]","[{'label': 'display', 'lat': 41.69330344761157...",41.693303,-87.839183,NaN,60477,IL,4eb3f7f2991165b761c201f5
2,Moraine Valley Community College- A280,College Classroom,9000 W College Pkwy,US,Palos Hills,United States,NaN,13176,"[9000 W College Pkwy, Palos Hills, IL 60465, U...","[{'label': 'display', 'lat': 41.69417125859782...",41.694171,-87.837638,NaN,60465,IL,4e60fd8f2271573ad6f7b8dd
3,Moraine Valley Community College Tutoring Center,Student Center,NaN,US,Palos Hills,United States,NaN,13674,"[Palos Hills, IL 60465, United States]","[{'label': 'display', 'lat': 41.68980259071242...",41.689803,-87.839830,NaN,60465,IL,4ff5bd1be4b0d61583815687
4,wilbur wright community college,None,NaN,US,Chicago,United States,NaN,14213,"[Chicago, IL 60630, United States]","[{'label': 'display', 'lat': 41.91168583359978...",41.911686,-87.696700,NaN,60630,IL,4fabe1bbe4b04b8d3023ae65
5,Moraine Valley Community College,College & University,NaN,US,Orland Park,United States,NaN,22554,"[Orland Park, IL, United States]","[{'label': 'display', 'lat': 41.6313427, 'lng'...",41.631343,-87.913117,NaN,NaN,IL,4e66551d1838ad3d107963e6
6,Richard J. Daley College,Community College,7500 S Pulaski Rd,US,Chicago,United States,NaN,3902,"[7500 S Pulaski Rd, Chicago, IL 60652, United ...","[{'label': 'display', 'lat': 41.75605576300383...",41.756056,-87.724051,NaN,60652,IL,4b71f4e2f964a52084662de3
7,Columbia College Chicago,University,623 S Wabash Ave,US,Chicago,United States,at E Harrison St,13558,"[623 S Wabash Ave (at E Harrison St), Chicago,...","[{'label': 'display', 'lat': 41.87416497086687...",41.874165,-87.625620,NaN,60605,IL,4f88bfcce4b05ae8883bd322
8,Marz Community Brewery,Brewery,3630 S Iron St,US,Chicago,United States,NaN,8119,"[3630 S Iron St, Chicago, IL 60609, United Sta...","[{'label': 'display', 'lat': 41.82786030288918...",41.827860,-87.659439,Bridgeport,60609,IL,5a91b043c824ae5f721195c1
9,Maria's Packaged Goods & Community Bar,Bar,960 W 31st St,US,Chicago,United States,at S Morgan St,9339,"[960 W 31st St (at S Morgan St), Chicago, IL 6...","[{'label': 'display', 'lat': 41.83809279514403...",41.838093,-87.651022,NaN,60608,IL,4c7db04610916dcb3e5d2a96


In [70]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) 


folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Midway Airport',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map